# Building a Digital Orrery

----

By Adam A Miller (Northwestern/CIERA/SkAI)  
09 Sept 2025

**Version 0.1**

(based on a problem originally written by Jeff Oishi [for the DSFP](https://github.com/LSSTC-DSFP/LSSTC-DSFP-Sessions/blob/main/Sessions/Session08/Day1/OOP_problem.ipynb))

In this problem you will construct a Digital Orrery. An [orrery](https://en.wikipedia.org/wiki/Orrery) is a mechanical model of the Solar System. Here, we will generalize this to anything that is mechanically similar to *the* solar system: a collection of things bound gravitationally. 

<img src="https://upload.wikimedia.org/wikipedia/commons/4/48/Grand_orrery_in_Putnam_Gallery%2C_2009-11-24.jpg" alt="Orrery" width="600"/>
(image: wikimedia)


In [54]:
import numpy as np
# import matplotlib.pyplot as plt

## Problem 1) Building a basic set of objects

Our first task is to map our problem onto a set of **objects** that we **instantiate** (that is, make **instances** of) in order to solve our problem.

Let's outline the scope of our problem.

A solar system exists in a Universe; here we can ignore the gravitational perturbation on the Solar System from the rest of the Universe (i.e., the Orrery can be treated as the only thing in the Universe). Our model will consist of a small number of bodies containing mass. It might also contain bodies without mass, so called "test particles."

The problem to be solved numerically is the gravitational N-body problem,

$$\ddot{\mathbf{r}}_i = -G\sum_{i \ne j} \frac{m_j \mathbf{r}_{ij}}{r_{ij}^3},$$

where $\mathbf{r}_{ij} \equiv \mathbf{r_i} - \mathbf{r_j}$. This task itself can be broken into two components: 

* the force calculation
* the ODE integrator to advance $\mathbf{r}_i$ and $\dot{\mathbf{r}}_i$ forward in time

**Problem 1a**

In disucssion with a classmate, sketch out a set of classes that you will need to complete this project. Don't worry about things like numerical integrators yet. 

Also, sketch out interfaces (start with the class constructor), but don't worry about writing code right now.

*Hint* – what is the minimal number of classes you will need to perform the calculation?

*write your response here*

To build the Orrery we will need two classes. One called `Body`, which can be used to hold instances of the different objects that are gravitationally interacting. The important properties for a body are its mass, position, velocity, and acceleration. 

We also need a `Universe` class, which will include all the bodies that are present within the Universe. 

**Problem 1b**

Using `r0` and `rdot0` below create an instance of the `Body` class. 

*Hint* – if you did not fully create the `Body` class in the previous problem do that here.

In [55]:
# complete
class Body(): 
    def __init__(self, mass, position, velocity):
        self.mass = mass
        self.position = position
        self.velocity = velocity


In [56]:
r0 = np.array([0.0,0.0,0.0])
rdot0 = np.array([0.0,0.0,0.0])

In [57]:
b = Body(1,r0, rdot0)

## Problem 2

Now, we code the numerical algorithms. We're going to do the most simple things possible: a *brute force* ("direct N-Body" if you're feeling fancy) force calculation, and a leapfrog time integrator.

The leapfrog scheme is an explicit, second order scheme given by

$$r_{i+1} = r_{i} + v_{i} \Delta t + \frac{\Delta t^2}{2} a_{i}$$

$$v_{i+1} = v_{i} + \frac{\Delta t}{2} (a_{i} + a_{i+1}),$$

where $\Delta t$ is the time step (which we'll just keep constant), and the subscript refers to the *iteration* number $i$. 

Note that this scheme requires a force update *in between* calculating $r_{i+1}$ and $v_{i+1}$. (In other words, the code should proceed a half time step to update the velocity of the bodies, then calculate the new position of the bodies using a full time step, compute the forces, and then proceed with an additional half time step to get the updated velocity.)

**Problem 2a** 

Write a method that implements the force integrator. Test it on simple cases:
 * two equal 1 $M_\odot$ objects in your universe, 1 AU apart
 * a $1\ M_\odot$ object and a $1\ M_{\oplus}$ object, 1 AU apart

In [58]:
class Universe(): 
    def __init__(self, body_list): 
        self.body_list = body_list
        self.size = len(self.body_list)
    
    def compute_forces(self): 
        G = 39.478 # in unit au^3 yr^-2 M_sun^-1
        n = self.size
        accel_list = np.zeros((n,3))
        for i in range(n):
            accel = 0
            body1 = self.body_list[i]
            for j in range(n):
                if not i == j:
                    body2 = self.body_list[j]
                    distance_vector = body1.position - body2.position
                    distance = np.linalg.norm(distance_vector)
                    accel += -G*body2.mass/distance**3 * distance_vector
            accel_list[i] = accel
        return accel_list


In [59]:
# test case 1 two equal mass bodies separated by 1 AU

# complete
# complete
sun1_r0 = np.array([0, 0, 0])
sun1_v0 = np.array([0, 0, 0])
sun1 = Body(1, sun1_r0, sun1_v0)
sun2_r0 = np.array([1, 0, 0])
sun2_v0 = np.array([0, 0, 0])
sun2 = Body(1, sun2_r0, sun2_v0)

universe_sun_sun = Universe([sun1, sun2])
force = universe_sun_sun.compute_forces()
print(force)
# complete

[[ 39.478   0.      0.   ]
 [-39.478   0.      0.   ]]


In [60]:
# test case 2 - sun + earth system separated by 1 AU

# complete
# complete
sun_r0 = np.array([0, 0, 0])
sun_v0 = np.array([0, 0, 0])
sun = Body(1, sun1_r0, sun1_v0)
earth_r0 = np.array([1, 0, 0])
earth_v0 = np.array([0, 0, 0])
earth = Body(1/333000, earth_r0, earth_v0)

universe_sun_earth = Universe([sun, earth])
force = universe_sun_earth.compute_forces()
print(force)

# complete

[[ 1.18552553e-04  0.00000000e+00  0.00000000e+00]
 [-3.94780000e+01  0.00000000e+00  0.00000000e+00]]


**Problem 2b**
Write the leapfrog integration as a method in the `Universe` class. Test it on one particle with no force (what should it do?)

In [ ]:
class Universe(): 
    # complete
    def __init__(self, body_list): 
        self.body_list = body_list
        self.size = len(self.body_list)
    
    def compute_forces(self): 
        G = 39.478 # in unit au^3 yr^-2 M_sun^-1
        n = self.size
        accel_list = np.zeros((n,3))
        for i in range(n):
            accel = 0
            body1 = self.body_list[i]
            for j in range(n):
                if not i == j:
                    body2 = self.body_list[j]
                    distance_vector = body1.position - body2.position
                    distance = np.linalg.norm(distance_vector)
                    accel += -G*body2.mass/distance**3 * distance_vector
            accel_list[i] = accel
        return accel_list
    
    def leapfrog_step(self, dt):
        n = self.size
        accel_list = self.compute_forces()
        for i in range(n): # Position Update
            body = self.body_list[i]
            accel = accel_list[i]
            body.position += body.velocity*dt + dt**2/2*accel
        accel_list = self.compute_forces() # Re-calculate the force for velocity update
        for i in range(n): # Velocity Update
            body = self.body_list[i]
            accel_new = accel_list[i]
            body.velocity += dt/2*(accel+accel_new)

    def move(self, time, steps):
        dt = time/steps
        for i in range(steps):
            self.leapfrog_step(dt)
 
        # complete
        # complete
        
        # complete

In [68]:
# Test case: One particle with no force

# complete
# complete
particle = Body(1, np.array([0.0, 0.5, 0.0]), np.array([0.1, 0.0, 0.0]))

universe_one_body = Universe([particle])
universe_one_body.move(3, 20)
print(particle.position)
print(particle.velocity)

# complete


[0.3 0.5 0. ]
[0.1 0.  0. ]


**Problem 2c** 
 
Wire it all up! Try a 3-body calculation of the Earth-Sun-Moon system. Try the Earth-Jupiter-Sun system! 

In [ ]:
# complete
# complete

# complete

n_steps = 1000
for step_num in range(n_steps): 
    u.leapfrog_step(3600) # 1 hour timestep

print(f'After {n_steps} hours, the bodies have now moved to:')
for b in u.bodies:
    print(f'The position of this body is {b.r}')

## Challenge Problem

* Construct a visualization method for the `Universe` class
* Read about the Fast Multipole Method (FMM) [here](https://math.nyu.edu/faculty/greengar/shortcourse_fmm.pdf) and implement one for the force calculation

In [ ]:
# good luck!